In [ ]:
#JJ McCauley + LOGAN KELSCH 
#TEST NN 1

#IMPORT LIBRARIES-------------------------------------------------------

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD
from keras.initializers import GlorotUniform
from keras.initializers import RandomNormal
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from funcs_data_preprocess import *

#hahaha dont turn this on with high epoch or else
#tf.config.experimental.set_memory_growth

#LOAD DATA FROM CSV-------------------------------------------------------

# Load the dataset
data = pd.read_csv('d1_test_.csv')
dataTRAIN = pd.read_csv('catted_1-8.csv')

#      'Dr1' 'Dr3' 'Mr1' 'Mr3' 
testFor = 'r30'
timeSteps = 10
tType = testFor[0]

data = data.drop(columns=['FT'])#,'FT.1'])
dataTRAIN = dataTRAIN.drop(columns=['FT','FT.1'])
#CALENDAR
#--------------------------------------
#SOLUTION------------------------------
match testFor:
    case 'r1':
        data = data.drop(columns=['r2','r3','r5','r10','r15','r30','r60'])
        dataTRAIN = dataTRAIN.drop(columns=['r2','r3','r5','r10','r15','r30','r60'])
    case 'r2':
        data = data.drop(columns=['r1','r3','r5','r10','r15','r30','r60'])
        dataTRAIN = dataTRAIN.drop(columns=['r1','r3','r5','r10','r15','r30','r60'])
    case 'r3':
        data = data.drop(columns=['r1','r2','r5','r10','r15','r30','r60'])
        dataTRAIN = dataTRAIN.drop(columns=['r1','r2','r5','r10','r15','r30','r60'])
    case 'r5':
        data = data.drop(columns=['r1','r2','r3','r10','r15','r30','r60'])
        dataTRAIN = dataTRAIN.drop(columns=['r1','r2','r3','r10','r15','r30','r60'])
    case 'r10':
        data = data.drop(columns=['r1','r2','r3','r5','r15','r30','r60'])
        dataTRAIN = dataTRAIN.drop(columns=['r1','r2','r3','r5','r15','r30','r60'])
    case 'r15':
        data = data.drop(columns=['r1','r2','r3','r5','r10','r30','r60'])
        dataTRAIN = dataTRAIN.drop(columns=['r1','r2','r3','r5','r10','r30','r60'])
    case 'r30':
        data = data.drop(columns=['r1','r2','r3','r5','r10','r15','r60'])
        dataTRAIN = dataTRAIN.drop(columns=['r1','r2','r3','r5','r10','r15','r60'])
    case 'r60':
        data = data.drop(columns=['r1','r2','r3','r5','r10','r15','r30'])
        dataTRAIN = dataTRAIN.drop(columns=['r1','r2','r3','r5','r10','r15','r30'])


#confirming X and Y features post training
Xfeatures = data.columns[:-1]
Yfeatures = data.columns[-1]
XTRAINfeatures = dataTRAIN.columns[:-1]
YTRAINfeatures = dataTRAIN.columns[-1]
print("TESTED FEATURES: ")
print(Xfeatures)
print("TESTING FOR: ")
print(Yfeatures)
print("TRAINED FEATURES: ")
print(XTRAINfeatures)
print("TRAINED FOR: ")
print(YTRAINfeatures)

# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

XTRAIN = dataTRAIN.iloc[:, :-1].values
yTRAIN = dataTRAIN.iloc[:, -1].values

X = normalize_from_tt_split(X, XTRAIN, 0.2)
XTRAIN = normalize_from_tt_split(XTRAIN, XTRAIN, 0.2)
X = reform_with_PCA_isolated(X, XTRAIN, 0.2, 3, 10)
X = reformat_to_lstm(X, timeSteps)

y = y[timeSteps:]
y = np.array(y)

XTRAIN = reformat_to_lstm(XTRAIN, timeSteps)
yTRAIN = yTRAIN[timeSteps:]
yTRAIN = np.array(yTRAIN)






t0930 = -0.32007092
t1045 = -0.141503587
t1200 = 0.03706375
t1600 = 10


X, y = remove_zero_mo_samples(X, y, timeSteps)
X, y = remove_extra_filter(X, y, timeSteps, t0930, t1045)

X = X[:,:,:-1]
#XTRAIN, yTRAIN = remove_zero_mo_samples(XTRAIN, yTRAIN)

print('X shape == {}.'.format(X.shape))
print('y shape == {}.'.format(y.shape))

from keras.saving import get_custom_objects
from keras.saving import register_keras_serializable
get_custom_objects().clear()
#CUSTOM LOSS 1_______________________________________________________________________________________________
from keras.src import ops
from keras.src.losses.loss import squeeze_or_expand_to_same_rank
@register_keras_serializable(name="skew_loss")
def skew_loss(y_true,y_pred,sFact=4):
    #return ops.mean(ops.square((y_pred-y_true)*(1+sFact*tf.cast(((y_true>0 & y_pred<y_true) | (y_true<0 & y_pred>y_true)),tf.float32))),axis=-1)
    y_pred = ops.convert_to_tensor(y_pred)
    y_true = ops.convert_to_tensor(y_true, dtype=y_pred.dtype)
    #y_true, y_pred = squeeze_or_expand_to_same_rank(y_true, y_pred)
    error = ops.subtract(y_pred, y_true)
    a = ops.convert_to_tensor(ops.cast(y_pred > 0,tf.float32), dtype=tf.float32)
    b = ops.convert_to_tensor(ops.cast(y_pred < 0,tf.float32), dtype=tf.float32)
    c = ops.convert_to_tensor(ops.cast(y_true >= y_pred,tf.float32), dtype=tf.float32)
    d = ops.convert_to_tensor(ops.cast(y_true <= y_pred,tf.float32), dtype=tf.float32)
    h = ops.convert_to_tensor(0.1, dtype=error.dtype)
    return ops.mean(
        ops.where(
            a*c+b*d==1,# or (b and d),
            h*ops.square(error),
            ops.square(error)
        ))


loaded_model = tf.keras.models.load_model('r30_PCA10_2.keras')#----------------------------------------------------------------------
#loaded_model.compile(optimizer='adam',
#                  loss='mse'
#                  ,metrics=['R2Score','root_mean_squared_error'])

#predicting the test set results
y_pred = loaded_model.predict(X) 

s_kws = {'s':2,'color':'maroon'}
l_kws = {'lw':1,'color':'maroon'}

import seaborn as sns
y_pred = np.squeeze(y_pred)
y = np.squeeze(y)
ys = pd.DataFrame({"y_pred":y_pred,"y_true":y})
#data.insert(1, "y_pred", y_pred, True)
# plot 1 with axes level-plot
'''
g = sns.lmplot(data=ys,x="y_pred", y="y_true", scatter_kws=s_kws,line_kws=l_kws)#, hue="MO")
plt.axis('tight')
plt.grid()
plt.title('Testing Outputs')
plt.xlabel('y_pred')
plt.ylabel(testFor)
plt.xlim(-.25,.25)
plt.ylim(-.25,.25)
plt.show()
'''
g = sns.lmplot(data=ys,x="y_pred", y="y_true", scatter_kws=s_kws, line_kws=l_kws)#,hue="MO")
plt.title('Zoomed in graph')
plt.grid()
y_min, y_max = ys['y_true'].min(), ys['y_true'].max()
plt.ylim(y_min, y_max)
x_center = np.mean(ys['y_pred'])
y_range = y_max - y_min
x_min = x_center - y_range / 2
x_max = x_center + y_range / 2
plt.xlim(x_min, x_max)
plt.show()

#DIRECTIONAL ACCURACY #DIRECTIONAL ACCURACY  #DIRECTIONAL ACCURACY  #DIRECTIONAL ACCURACY  #DIRECTIONAL ACCURACY  
specGuess = .35
frac = 100
fracf = 1/frac
specAcc, locSpecAcc = [], []
valCount, locValCount = [], []
locMean = []
locStd = []
sameDirAvg, sameDirMax, sameDirMin = [], [], []
diffDirAvg, diffDirMax, diffDirMin = [], [], []
locSameDirAvg, locSameDirMax, locSameDirMin = [], [], []
locDiffDirAvg, locDiffDirMax, locDiffDirMin = [], [], []
for v in range(0,int(specGuess*frac)):
    tp, fp, tn, fn, tpL, fpL, tnL, fnL = 0, 0, 0, 0, 0, 0, 0, 0
    valCnt = 0
    locValCnt = 0
    allTmp = []
    sameTmp, diffTmp = [], []
    locSameTmp, locDiffTmp = [], []
    y_pred = ys['y_pred']
    y_test = ys['y_true']
    for i in range(len(y_pred)):
        if(y_pred[i]>=(v/frac)):
            valCnt+=1
            if(y_test[i]>0):
                tp+=1
                sameTmp.append(abs(y_test[i]))
            if(y_test[i]<0):
                fp+=1
                diffTmp.append(abs(y_test[i]))
            if(y_pred[i]<(v/frac)+(1/frac)):
                locValCnt+=1
                if(y_test[i]>0):
                    tpL+=1
                    locSameTmp.append(abs(y_test[i]))
                if(y_test[i]<0):
                    fpL+=1
                    locDiffTmp.append(abs(y_test[i]))
        if(y_pred[i]<=-(v/frac)):
            valCnt+=1
            if(y_test[i]<0):
                tn+=1
                sameTmp.append(abs(y_test[i]))
            if(y_test[i]>0):
                fn+=1
                diffTmp.append(abs(y_test[i]))
            if(y_pred[i]>-(v/frac)-(1/frac)):
                locValCnt+=1
                if(y_test[i]<0):
                    tnL+=1
                    locSameTmp.append(abs(y_test[i]))
                if(y_test[i]>0):
                    fnL+=1
                    locDiffTmp.append(abs(y_test[i]))
    if((tp+fp+tn+fn)<1):
        break
    

    if(len(sameTmp)>0 and len(diffTmp)>0):
        allTmp = locSameTmp.copy()
        locDiffTmp = np.multiply(locDiffTmp, -1)
        for i in range(len(locDiffTmp)):
            allTmp.append(locDiffTmp[i])
        locMean.append(np.average(allTmp))
        locStd.append(np.std(allTmp))
        locDiffTmp = np.multiply(locDiffTmp, -1)
    else:
        locMean.append(locMean[-1])
        locStd.append(locStd[-1])

    if(len(locSameTmp)!=0):
        sameDirAvg.append(np.average(sameTmp))
        sameDirMax.append(np.max(sameTmp))
        sameDirMin.append(np.min(sameTmp))
        locSameDirAvg.append(np.average(locSameTmp))
        locSameDirMax.append(np.max(locSameTmp))
        locSameDirMin.append(np.min(locSameTmp))
    else:
        sameDirAvg.append(0.0)
        sameDirMax.append(0.01)
        sameDirMin.append(0.01)
        locSameDirAvg.append(0.01)
        locSameDirMax.append(0.01)
        locSameDirMin.append(0.01)
    if(len(locDiffTmp)!=0):
        diffDirAvg.append(np.average(diffTmp))
        diffDirMax.append(np.max(diffTmp))
        diffDirMin.append(np.min(diffTmp))
        locDiffDirAvg.append(np.average(locDiffTmp))
        locDiffDirMax.append(np.max(locDiffTmp))
        locDiffDirMin.append(np.min(locDiffTmp))
    else:
        diffDirAvg.append(0.01)
        diffDirMax.append(0.01)
        diffDirMin.append(0.01)
        locDiffDirAvg.append(0.01)
        locDiffDirMax.append(0.01)
        locDiffDirMin.append(0.01)
    directionalAccuracy = ((tp+tn)/(tp+fp+tn+fn))*10000//1/100
    specAcc.append(directionalAccuracy)
    valCount.append(valCnt)
    if((tpL+fpL+tnL+fnL) > 0):
        localDirectionalAccuracy = ((tpL+tnL)/(tpL+fpL+tnL+fnL))*10000//1/100
        locSpecAcc.append(localDirectionalAccuracy)
        locValCount.append(locValCnt)
    else:
        locSpecAcc.append(locSpecAcc[-1])
        locValCount.append(0)
    #print(f'Directional Accuracy >(+/-){v/2}:\t',directionalAccuracy5guess)
#print('Directional Accuracy:\t\t',directionalAccuracy)
valCount = [x / valCount[0] * 100 for x in valCount]
locValCount = [x / locValCount[0] * 100 for x in locValCount]

std2 = np.multiply(locStd, 2)
std0 = np.multiply(locStd, 0)
yerr = [std2,std0]
plt.title('Mean Move and Standard Deviations per Predicted Value')
plt.xlabel('> (+/-) Prediction Value')
plt.errorbar(np.arange(0,(1/frac)*len(specAcc),(1/frac)), locMean, yerr=yerr, capsize=3, ecolor = "gray", elinewidth=1, markeredgewidth='1',markersize=2)
plt.errorbar(np.arange(0,(1/frac)*len(specAcc),(1/frac)), locMean, locStd, capsize=3, ecolor = "black", elinewidth=1, markeredgewidth='1',markersize=2)
plt.axhline(0,color='black')
plt.grid()
plt.show()

plt.figure(figsize=(18, 9))
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), specAcc, 'maroon', label='Directional Accuracy')
plt.scatter(np.arange(0,(1/frac)*len(specAcc),(1/frac)), specAcc, s=5, color='maroon')
plt.plot(np.arange(0,(1/frac)*len(locSpecAcc),(1/frac)), locSpecAcc, 'red', label='Directional Accuracy')
plt.scatter(np.arange(0,(1/frac)*len(locSpecAcc),(1/frac)), locSpecAcc, s=5, color='red')
plt.title(f'Directional Accuracy Percentage Outside of given Predicted Value\n(With Percentage of {len(X)} Samples)')
plt.xlabel('> (+/-) Prediction Value')
plt.ylabel('Percent')
plt.grid()
plt.legend(['Directional Accuracy','S- Samples\nA- Accuracy','Local Directional Accuracy'], loc='upper left')
for x, y in zip(np.arange(0,(1/frac)*len(specAcc),(1/frac)), specAcc):
    plt.annotate(f'{float(valCount[int(x*frac)]*10//1/10)}%S\n{float(specAcc[int(x*frac)])}%A', color='black', xy=(x, y-2), fontsize=8)
plt.show()

plt.figure(figsize=(16, 8))
plt.subplot(1,2,1)
plt.title('Average Move by Directional Accuracy')
plt.xlabel('> (+/-) Prediction Value')
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), locSameDirAvg, '#bbbbbb', label='Directional Accuracy')
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), sameDirAvg, 'maroon', label='Directional Accuracy')
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), locDiffDirAvg, '#bbbbbb', label='Directional Accuracy')
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), diffDirAvg, 'purple', label='Directional Accuracy')
plt.grid()
plt.legend(['Accurate','Accurate - Local','Inaccurate','Inaccurate - Local'], loc='upper left')
plt.subplot(1,2,2)
plt.title('P/L Ratio')
plt.xlabel('> (+/-) Prediction Value')
plt.grid()
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)),np.divide(sameDirAvg,diffDirAvg), 'blue')
plt.legend(['P/L Ratio'], loc='upper left')
plt.ylim(0,6)
plt.show()
plt.figure(figsize=(16, 8))
plt.subplot(1,2,1)
plt.title('Highest and Lowest of\nDirectionally Accurate Moves')
plt.xlabel('> (+/-) Prediction Value')
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), sameDirMax, 'maroon', label='Directional Accuracy')
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), sameDirMin, 'maroon', label='Directional Accuracy')
plt.grid()
plt.subplot(1,2,2)
plt.title('Highest and Lowest of\nDirectionally Inaccurate Moves')
plt.xlabel('> (+/-) Prediction Value')
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), diffDirMax, 'purple', label='Directional Accuracy')
plt.plot(np.arange(0,(1/frac)*len(specAcc),(1/frac)), diffDirMin, 'purple', label='Directional Accuracy')
plt.grid()
plt.show()

In [ ]:
data.corr()
#print(X.shape)

In [ ]:
catReform = pd.read_csv('catted_12day.csv')
print(y_pred)
print((y_pred.shape))
#y_pred = y_pred.to_numpy()
print(type(y_pred))
y_pred = np.insert(y_pred, 0, 0, axis=0)
y_pred = np.insert(y_pred, 0, 0, axis=0)
y_pred = np.insert(y_pred, 0, 0, axis=0)
y_pred = np.insert(y_pred, 0, 0, axis=0)
y_pred = np.insert(y_pred, 0, 0, axis=0)
#y_pred = np.append(y_pred, -1, 0, axis=0)
y_pred = pd.Series(y_pred)
catReform['Dr3_Model'] = y_pred
print(catReform.tail(5))
catReform.to_csv('catted_12day_Dr3Model.csv')

In [ ]:
y_pred = pd.Series(y_pred)
catReform['Dr3_Model'] = y_pred
print(catReform.tail(5))
catReform.to_csv('catted_1_.csv')

In [ ]:
catReform.to_csv('catted_12day_Dr3Model.csv',index=False)

In [ ]:
dataPolish = pd.read_csv('catted_12day_Dr3Model.csv')
cols = ['FT','vel5','vel10','vel15','vel30','vel60','acc5','acc10','acc15','acc30','acc60','stoch12','stochDiff6012','RSIhl_diff','RSIhl_diffROC','vol','vol10','vol15','vol30','vol60','volD10','volD15','volD30','volD60','vpm5','vpm10','vpm15','vpm30','vpm60','ToD','DoW','MO','Dr3_Model','Dr3','Dc1','Dc3','Dr1','Mc1','Mc3','Mr1','Mr3']
dataPolish = dataPolish.reindex(columns=cols)
#dataPolish = dataPolish.drop(columns=['YM_diff','NQ_diff','volNQdiff','volYMdiff'])
dataPolish.to_csv('catted_12day_Dr3Model.csv',index=False)

In [ ]:
sns.pairplot(data, diag_kind="kde", plot_kws={'alpha':0.2})  # 'kde' for smoother diagonal plots
plt.suptitle('Pairplot of Feature Relationships', y=1.02)
plt.show()

In [ ]:
print(data.describe())  # Summary statistics
print(data.info())      # Column info, data types, and null counts
print(dataTRAIN.describe())  # Summary statistics
print(dataTRAIN.info())      # Column info, data types, and null counts

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()

X_ = data.iloc[:, :-1].values
y_ = data.iloc[:, -1].values

X_new = pca.fit_transform(X_)

print(pca.get_covariance())

In [ ]:
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

In [ ]:
plt.figure(figsize=(6, 4))

plt.bar(range(36), explained_variance, alpha=0.5, align='center',
        label='individual explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.tight_layout()



In [ ]:
from sklearn.neighbors import KNeighborsRegressor


pca = PCA(n_components = 16)
pca.fit(X_)
X_new = pca.transform(X_)


retained_variance = np.sum(pca.explained_variance_ratio_)

# Step 3: Calculate percent data loss
data_loss = (1 - retained_variance) * 100

print(f"Retained Variance: {retained_variance * 100:.2f}%")
print(f"Data Loss: {data_loss:.2f}%")

X_train_new, X_test_new, y_train, y_test = \
    train_test_split(X_new, y_, test_size = 0.1, random_state=0, shuffle=False)

knn_pca = KNeighborsRegressor(64)
knn_pca.fit(X_train_new,y_train)
print("Train score after PCA", knn_pca.score(X_train_new,y_train), "%")
print("Test score after PCA", knn_pca.score(X_test_new,y_test), "%")

In [ ]:
#scaler1.fit(X_)
#X_ = scaler1.transform(X_)
scaler1.fit(X_)
X_ = scaler1.transform(X_)

data_loss = []
for i in range(16):
    pca = PCA(n_components = i+1)
    pca.fit(X_)
    X_new = pca.transform(X_)
    retained_variance = np.sum(pca.explained_variance_ratio_)
    data_loss.append(retained_variance)

plt.plot(range(16), data_loss, color='maroon')
plt.grid()
plt.ylim(.9,1)
plt.show()
